In [29]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50V2
from sklearn.utils import shuffle

nb_classes = 10
nb_initial_epoch = 1
nb_epoch = 1
batch_size = 128

# Load and preprocess the data
(X_train_all, y_train_all), (X_test, y_test) = cifar10.load_data()

X_train_all, y_train_all = shuffle(X_train_all, y_train_all)

X_pool = X_train_all[10000:50000]
y_pool = y_train_all[10000:50000]

X_train_all = X_train_all[:10000]
y_train_all = y_train_all[:10000]

X_train = X_train_all[:2500]
y_train = y_train_all[:2500]

X_train = X_train.astype('float32') / 255
X_pool = X_pool.astype('float32') / 255
X_test = X_test.astype('float32') / 255

Y_train = to_categorical(y_train, nb_classes)
Y_pool = to_categorical(y_pool, nb_classes)
Y_test = to_categorical(y_test, nb_classes)

# Define the ResNet50V2 model
def create_model():
    base_model = ResNet50V2(include_top=False, input_shape=(32, 32, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x, training=True)
    predictions = Dense(nb_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

    return model

# Train the initial model
model = create_model()
model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_initial_epoch, verbose=1)


# Deep Bayesian Active Learning Loop
n_queries = 10
n_samples_to_acquire = 1000

# Reduce the number of MC samples
n_mc_samples = 20

for query in range(n_queries):
    print("Query:", query + 1)

    # Calculate uncertainties using MC Dropout
    uncertainties = []

    for i in range(len(X_pool)):
        mc_samples = np.array([model(X_pool[i:i + 1], training=True) for _ in range(n_mc_samples)])
        entropy = -np.mean(np.sum(mc_samples * np.log(mc_samples + 1e-10), axis=-1), axis=0)
        uncertainties.append(entropy)

    # Acquire new samples based on uncertainties
    idx_to_acquire = np.argsort(uncertainties)[-n_samples_to_acquire:]

    X_acquired = X_pool[idx_to_acquire]
    Y_acquired = Y_pool[idx_to_acquire]

    X_train = np.concatenate([X_train, X_acquired], axis=0)
    Y_train = np.concatenate([Y_train, Y_acquired], axis=0)

    # Remove acquired samples from the pool
    X_pool = np.delete(X_pool, idx_to_acquire, axis=0)
    Y_pool = np.delete(Y_pool, idx_to_acquire, axis=0)

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_6[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 18, 18, 64)   0           ['conv1_conv[0][0]']             
                                                                                           

20/79 [======>.......................] - ETA: 21s - loss: 2.5363 - accuracy: 0.1406

KeyboardInterrupt: 